In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from pinecone import Pinecone, ServerlessSpec

In [18]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeConfigurationError: You haven't specified an Api-Key.

In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great lecturer, but the exams were tough. Really makes you think.'},
 {'professor': 'Prof. Emily Davis',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Passionate and knowledgeable. Her enthusiasm is contagious!'},
 {'professor': 'Dr. Michael Johnson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Clear explanations, but sometimes goes too fast. Needs better office hours.'},
 {'professor': 'Prof. Sarah Lee',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Not very approachable and the labs were poorly organized.'},
 {'professor': 'Dr. David Brown',
  'subject': 'History',
  'stars': 4,
  'review': 'Engaging lectures with lots of stories, but too many readings.'},
 {'professor': 'Prof. Jessica Wilson',
  'subject': 'English Literature',
  'stars': 5,
  'review': 'Amazing professor! She made the readings so much fun.'},
 {'professor': 'Dr. Thomas Miller',
  'subject': 'Computer Science',

In [10]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

processed_data = []

for review in data['reviews']:
    # Tokenize and encode the input text
    inputs = tokenizer(review['review'], return_tensors='pt', truncation=True, padding=True)

    # Get the embeddings from BERT
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

    # Append the processed data
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\ryanr\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\ryanr\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\ryanr\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

In [11]:
processed_data[0]

{'values': [0.09180483222007751,
  -0.08138332515954971,
  0.30952662229537964,
  0.08419371396303177,
  0.15957149863243103,
  0.17650800943374634,
  0.38355690240859985,
  0.9112775921821594,
  -0.12550440430641174,
  0.019556311890482903,
  0.4381130635738373,
  -0.3467217981815338,
  0.4405198097229004,
  0.05128512531518936,
  -0.25680074095726013,
  0.008378196507692337,
  0.09942557662725449,
  -0.48001644015312195,
  -0.11416517198085785,
  0.12776248157024384,
  0.3081728518009186,
  0.03633446991443634,
  -0.03008241392672062,
  0.3343167006969452,
  0.39704111218452454,
  0.11332020163536072,
  0.045016732066869736,
  -0.12389735132455826,
  -0.24031652510166168,
  -0.3751086890697479,
  0.3078579306602478,
  0.10470649600028992,
  -0.2358105480670929,
  0.11039996147155762,
  0.22769348323345184,
  -0.3219457268714905,
  -0.29360413551330566,
  0.4490981996059418,
  -0.4192180633544922,
  0.010694311931729317,
  -0.4947243332862854,
  0.018067043274641037,
  0.0743677020072

In [16]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

upserted_count: 20

In [1]:
index.describe_index_stats()

NameError: name 'index' is not defined

In [19]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
import os
import json

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
   name="rag",
   dimension=768,
   metric="cosine",
   spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

# Load the review data
data = json.load(open("reviews.json"))

processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
   response = client.embeddings.create(
       input=review['review'], model="text-embedding-3-small"
   )
   embedding = response.data[0].embedding
   processed_data.append(
       {
           "values": embedding,
           "id": review["professor"],
           "metadata":{
               "review": review["review"],
               "subject": review["subject"],
               "stars": review["stars"],
           }
       }
   )

# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
   vectors=processed_data,
   namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())


PineconeConfigurationError: You haven't specified an Api-Key.